In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
from csnet.training.dataset import run_inference

torch.set_default_dtype(torch.float32)

In [ ]:
model_path = "/home/angiod@usi.ch/CSnet/results/SHIFTX2/production-carbons/best_model.pth"
test_regex = "/storage_common/angiod/NMR/SHIFTX2/npz/test/*.npz"
device = "cuda:2"

In [ ]:
run_inference(
    model_path=model_path,
    test_regex=test_regex,
    device=device,
    output_dir='../inference',
)

In [ ]:
import numpy as np
import glob
from plot import plot_violin, plot_distribution, plot_violindist, plot_inference

atoms_filters = ['*_HE_*', '*_HE1_*', '*_HE2_*']

for atoms_filter in atoms_filters:
    errors = []
    for filename in glob.glob('../inference/*.npz'):
        dataset = np.load(filename, allow_pickle=True)
        fltr = np.all(~np.isnan(dataset["chemical_shifts"]), axis=0)
        res_fltr, atom_fltr, resid_fltr = atoms_filter.split('_')
        if res_fltr != '*':
            fltr = fltr * np.isin(dataset['atom_resnames'], res_fltr.split('-'))
        if atom_fltr != '*':
            fltr = fltr * np.isin(dataset['atom_names'].flatten(), atom_fltr.split('-'))
        if resid_fltr != '*':
            fltr = fltr * np.isin(dataset['atom_resnumbers'], [int(x) for x in resid_fltr.split('-')])
        atom_names = dataset['atom_fullnames'][fltr]

        chemical_shifts      = dataset["chemical_shifts"][:, fltr]
        chemical_shifts_pred = dataset["chemical_shifts_pred"][:, fltr, 0]

        errors.append(chemical_shifts_pred - chemical_shifts)

        # plot_inference(atom_names, chemical_shifts_pred, chemical_shifts, figsize=(20,16))
        # plot_distribution(atom_names, chemical_shifts_pred, [filename], 'Chemical Shifts', figsize=(20,8), overlap_same=False)
        # plot_violin(atom_names, [chemical_shifts_pred, chemical_shifts], [filename, filename+'TRUE'], 'Chemical Shifts', figsize=(60,16), overlap_same=False)
        # plot_violindist(chemical_shifts_pred, [filename], 'Chemical Shifts', figsize=(20,8))

    errors = np.concatenate(errors, axis=-1)
    import matplotlib.pyplot as plt

    a, b, c = plt.hist(errors[0], bins=40, edgecolor='#e0e0e0', linewidth=0.5, alpha=0.5, label=atoms_filter) # facecolor='#2ab0ff', 
    plt.vlines(np.abs(errors[0]).mean(), [0], [a.max()], color='k', linestyle='--', lw=1)
plt.legend()